<a href="https://colab.research.google.com/github/ahmedsaadalvi/Image-Classification/blob/main/Creating_Image_classification_API_with_TensorFlow_Serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0   2688      0  0:00:01  0:00:01 --:--:--  2690
OK


In [2]:
!apt-get update & apt-get install tensorflow-model-server

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Reading package lists... Done
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Building dependency tree... Done
Reading state information... Done
Fetched 229 kB in 2s (99.7 kB/s)
The following NEW packages will be insta

In [3]:
!pip install tensorflow-gpu==1.13.1

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.13.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==1.13.1


In [4]:
!pip install requests

In [6]:
import os
import json
import random
import requests
import subprocess
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10

%matplotlib inline

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

X_train = X_train / 255.0
X_test = X_test / 255.0

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[32, 32, 3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

model.fit(X_train,
          y_train,
          batch_size=128,
          epochs=10)

391/391 [==============================] - 225s 564ms/step - loss: 1.4876 - sparse_categorical_accuracy: 0.4629
Epoch 2/10
391/391 [==============================] - 217s 555ms/step - loss: 1.0212 - sparse_categorical_accuracy: 0.6392
Epoch 3/10
391/391 [==============================] - 212s 541ms/step - loss: 0.8315 - sparse_categorical_accuracy: 0.7085
Epoch 4/10
391/391 [==============================] - 211s 541ms/step - loss: 0.7134 - sparse_categorical_accuracy: 0.7502
Epoch 5/10
391/391 [==============================] - 211s 540ms/step - loss: 0.6226 - sparse_categorical_accuracy: 0.7833
Epoch 6/10
391/391 [==============================] - 211s 540ms/step - loss: 0.5367 - sparse_categorical_accuracy: 0.8119
Epoch 7/10
391/391 [==============================] - 211s 540ms/step - loss: 0.4581 - sparse_categorical_accuracy: 0.8395
Epoch 8/10
391/391 [==============================] - 211s 540ms/step - loss: 0.3877 - sparse_categorical_accuracy: 0.8639
Epoch 9/10
391/391 [=======

In [7]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 13s 40ms/step - loss: 0.9413 - sparse_categorical_accuracy: 0.7443


In [8]:
print("Test accuracy is {}".format(test_accuracy))

Test accuracy is 0.7443000078201294


In [9]:
MODEL_DIR = "model/"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
export_path

if os.path.isdir(export_path):
    !rm -r {export_path}

tf.saved_model.save(model, export_path)

In [12]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

!nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" > server.log 2>&1 &

!tail server.log


In [13]:
random_image = np.random.randint(0, len(X_test))

data = json.dumps({"signature_name":"serving_default", "instances":[X_test[random_image].tolist()]})
data

'{"signature_name": "serving_default", "instances": [[[[0.7098039215686275, 0.803921568627451, 0.5568627450980392], [0.6588235294117647, 0.7607843137254902, 0.4823529411764706], [0.6666666666666666, 0.7725490196078432, 0.47058823529411764], [0.6823529411764706, 0.792156862745098, 0.49019607843137253], [0.6784313725490196, 0.7843137254901961, 0.48627450980392156], [0.6705882352941176, 0.7647058823529411, 0.4549019607843137], [0.6313725490196078, 0.7098039215686275, 0.39215686274509803], [0.6666666666666666, 0.7568627450980392, 0.43137254901960786], [0.6313725490196078, 0.7411764705882353, 0.4117647058823529], [0.6627450980392157, 0.7607843137254902, 0.44313725490196076], [0.6784313725490196, 0.7568627450980392, 0.4588235294117647], [0.6862745098039216, 0.7647058823529411, 0.47843137254901963], [0.6745098039215687, 0.7725490196078432, 0.49411764705882355], [0.6274509803921569, 0.7215686274509804, 0.43529411764705883], [0.6941176470588235, 0.7725490196078432, 0.47843137254901963], [0.6980

In [15]:
data = '{"signature_name": "serving_default", "instances": [{"input_image": your_input_data}]}'

headers = {"content-type": "application/json"}

url = "http://localhost:8501/v1/models/cifar10:predict"

json_response = requests.post(url=url, data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']
predictions

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/cifar10:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7da4b21e7f70>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
plt.imshow(X_test[random_image])

In [ ]:
class_names[np.argmax(predictions[0])]

In [ ]:
specific_json_response = requests.post(url="http://localhost:8501/v1/models/cifar10/versions/1:predict", data=data, headers=headers)

specific_json_response